# churn_location_and_rating.ipynb

## First, install the enviorment

In [1]:
# 1) First: install Java, Spark and and run a local Spark session by just running this on Google Colab:
!apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null   # !apt-get --> install java
#!wget -q https://dlcdn.apache.org/spark/spark-3.3.4/spark-3.3.4-bin-hadoop3.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz
!tar xf spark-3.5.1-bin-hadoop3.tgz  # !tar --> like unzip
!pip install -q findspark  # !pip  --> instal a package, we cant import a library without installing it first, most libraries that we used were already installed
# This are INSTALLATION COMMANDS IN LINUX that we run in our collab space, it's similar to downloading programs an installing them on our computers
# installs Apache Spark 3.1.2, Java 8, and Findspark, a library that makes it easy for Python to find Spark


# 2) Second: set the locations where Spark and Java are installed to let know Colab where to find it.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.1-bin-hadoop3"


# 3) Third: import spark libraries and use them
import findspark
findspark.init("spark-3.5.1-bin-hadoop3") # SPARK_HOME
from pyspark.sql import SparkSession
# Create the session - We need to remember to close it at the end
# The session is basically our connection to Spark layer in the Hadoop ecosystem
spark = SparkSession.builder.master("local[*]").getOrCreate()
from pyspark.sql import Row
from pyspark.sql import functions


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

<br>
<br>
<hr class="dashed">
<br>
<br>

## Establishing connection between "Colab" to "Google Drive"

In [2]:
# Code for connecting our google drive to this collab notebook
from google.colab import drive
drive.mount('/content/drive')

# After you finish, look at the menu (gui to the left) and check under "files" that your "drive" folder was added.

Mounted at /content/drive


In [3]:
path = "/content/drive/MyDrive/BDA RONIT/My_Spark_Ronit"
!ls "/content/drive/MyDrive/BDA RONIT/My_Spark_Ronit"  # Check what files we have in the path


 appl_stock.csv
 churn_location_and_rating.csv
 churn_location_and_rating.ipynb
 churn_location_and_rating_new.ipynb
 County_Rating.csv
 NORTHWND_csv
 part-00000-d3719948-a574-419c-a00e-0933af18e86b-c000.csv
'part C'
 people.json
'PySpark UDF with PyCountry Example.ipynb'
 sales_info.csv
'spark streaming'
 walmart_stock.csv


<br>
<br>
<hr class="dashed">
<br>
<br>

## Import necessary ibraries

In [4]:
from pyspark.sql.types import (StructField, IntegerType,
                               StringType, StructType)
from pyspark.sql.functions import when
from pyspark.sql.functions import countDistinct, avg
from pyspark.sql.functions import format_number
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.functions import concat,col
import re
from pyspark.sql.functions import udf
!pip install pycountry
import pycountry
from pyspark.sql import Row
from pyspark.sql import functions





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 15.7 MB/s eta 0:00:00


<br>
<br>
<hr class="dashed">
<br>
<br>

## Load the necessary tables from the csv files to Spark DataFtrame

In [5]:
datapath = path + '/churn_location_and_rating.csv'

df_emp = spark.read.csv(datapath,header=True,inferSchema=True)

In [6]:
def clean_text(text):
    return re.sub(r'[^A-Za-z ]+', '', text).strip().lower()

clean_text_udf = udf(lambda x:clean_text(x), StringType())

In [7]:
df_emp = df_emp.withColumn('CustomerID', split(df_emp['customerID;Location;Rating'], ';').getItem(0)) \
               .withColumn('City', split(df_emp['customerID;Location;Rating'], ';').getItem(1)) \
               .withColumn('State', split(df_emp['customerID;Location;Rating'], ';').getItem(2)) \
               .withColumn('Rating', split(df_emp['customerID;Location;Rating'], ';').getItem(3)) \
               .withColumn('Country', split(df_emp['customerID;Location;Rating'], ';').getItem(4))

df_emp = df_emp.drop('customerID;Location;Rating')

In [8]:
df_emp.count()
df_emp.dropDuplicates().count()
print(f'The dataset contains {df_emp.count()} customers without duplicate')

The dataset contains 24858 customers without duplicate


In [9]:
column=['City','State','Country' ]
for c in column:
  x=df_emp.where(df_emp[c].contains('n/a')).count()
  y=df_emp.where(df_emp[c]=='').count()
  df_emp = df_emp.withColumn(c, regexp_replace(col(c), 'n/a', ''))
  df_emp=df_emp.withColumn(c,clean_text_udf(col(c)))
  df_emp = df_emp.withColumn(c, regexp_replace(col(c), '"', ''))
  df_emp = df_emp.withColumn(c, when(col(c) == '', 'Unknown').otherwise(col(c)))
  print(f'{c} there is {x} rows with string n/a and {y} rows with empty string')


City there is 41 rows with string n/a and 9 rows with empty string
State there is 1339 rows with string n/a and 0 rows with empty string
Country there is 4 rows with string n/a and 7 rows with empty string


In [10]:
column=['Rating','CustomerID' ]
for c in column:
  df_emp = df_emp.withColumn(c, regexp_replace(col(c), '"', ''))

df_emp = df_emp.withColumn('Rating', col('Rating').cast('int'))
null_Rating=df_emp.where(col('Rating').isNull()).count()
print(f'There is {null_Rating} with null')

There is 2 with null


In [11]:
df_emp=df_emp.where(df_emp.Rating.isNotNull())
count=df_emp.count()
print(f'The dataset contains {count} customers without duplicate')

The dataset contains 24856 customers without duplicate


In [12]:
df_emp.groupBy('Rating').count().show()

+------+-----+
|Rating|count|
+------+-----+
|     1| 2449|
|     6| 2530|
|     3| 2491|
|     5| 2486|
|     9| 2529|
|     4| 2510|
|     8| 2409|
|     7| 2574|
|    10| 2347|
|     2| 2531|
+------+-----+



In [13]:
df_emp.show()

+----------+--------------+---------------+------+--------------+
|CustomerID|          City|          State|Rating|       Country|
+----------+--------------+---------------+------+--------------+
|    id7044|           nyc|       new york|     9|           usa|
|    id7045|      stockton|     california|     5|           usa|
|    id7046|        moscow|yukon territory|     5|        russia|
|    id7047|         porto|         vngaia|     9|      portugal|
|    id7048|   farnborough|          hants|     8|united kingdom|
|    id7049|  santa monica|     california|    10|           usa|
|    id7050|    washington|             dc|     3|           usa|
|    id7051|       timmins|        ontario|     2|        canada|
|    id7052|    germantown|      tennessee|    10|           usa|
|    id7053|      albacete|      wisconsin|    10|         spain|
|    id7054|     melbourne|       victoria|     3|     australia|
|    id7055|    fort bragg|     california|     6|           usa|
|    id705

# Pycountry

In [14]:
def findCountry(country_name):
  try:
    # Convert to title case to match pycountry's expectations
    country_name_title = country_name.title()
    # Try to find the country by name
    country = pycountry.countries.get(name=country_name_title)
    if country:
        return country.name
    # Try to find the country by alpha_3 code in uppercase
    country = pycountry.countries.get(alpha_3=country_name.upper())
    if country:
        return country.name
    country = pycountry.countries.get(official_name=country_name.upper())
    if country:
        return country.name
    country = pycountry.countries.get(alpha_2=country_name.upper())
    if country:
        return country.name
    country = pycountry.countries.get(common_name=country_name.upper())
    if country:
        return country.name

    try:
      country_fuzzy = pycountry.countries.search_fuzzy(country_name)
      if country_fuzzy:
        return country_fuzzy[0].name
    except:
      return "Unknown"

    # If none of the lookups work, return "Unknown"
    return "Unknown"
  except: # Exception as e
    return "Unknown"


findCountryUDF = udf(lambda x: findCountry(x),StringType())

In [15]:
df_emp=df_emp.select("CustomerID", "City",'State','Rating', findCountryUDF(col("Country")).alias("Country"),findCountryUDF(col("State")).alias("Country_new"))
df_emp.show()

+----------+--------------+---------------+------+------------------+--------------+
|CustomerID|          City|          State|Rating|           Country|   Country_new|
+----------+--------------+---------------+------+------------------+--------------+
|    id7044|           nyc|       new york|     9|     United States| United States|
|    id7045|      stockton|     california|     5|     United States| United States|
|    id7046|        moscow|yukon territory|     5|Russian Federation|       Unknown|
|    id7047|         porto|         vngaia|     9|          Portugal|       Unknown|
|    id7048|   farnborough|          hants|     8|    United Kingdom|       Unknown|
|    id7049|  santa monica|     california|    10|     United States| United States|
|    id7050|    washington|             dc|     3|     United States|United Kingdom|
|    id7051|       timmins|        ontario|     2|            Canada|        Canada|
|    id7052|    germantown|      tennessee|    10|     United Sta

In [16]:
df_emp = df_emp.withColumn(
    "Final_Country",when(col("Country") == "Unknown", col("Country_new")).otherwise(col("Country")))

In [17]:
df_emp.createOrReplaceTempView("emp_table")

query = """
SELECT
    CustomerID,
    City,
    State,
    Rating,
    CASE
        WHEN Country = 'Unknown' THEN Country_new
        ELSE Country
    END AS Country
FROM emp_table
"""

df_emp = spark.sql(query)

df_emp.show()

+----------+--------------+---------------+------+------------------+
|CustomerID|          City|          State|Rating|           Country|
+----------+--------------+---------------+------+------------------+
|    id7044|           nyc|       new york|     9|     United States|
|    id7045|      stockton|     california|     5|     United States|
|    id7046|        moscow|yukon territory|     5|Russian Federation|
|    id7047|         porto|         vngaia|     9|          Portugal|
|    id7048|   farnborough|          hants|     8|    United Kingdom|
|    id7049|  santa monica|     california|    10|     United States|
|    id7050|    washington|             dc|     3|     United States|
|    id7051|       timmins|        ontario|     2|            Canada|
|    id7052|    germantown|      tennessee|    10|     United States|
|    id7053|      albacete|      wisconsin|    10|             Spain|
|    id7054|     melbourne|       victoria|     3|         Australia|
|    id7055|    fort

 **take a long time to run!!!**

In [18]:
x=df_emp.select('Country')
Unknown=x.filter("Country='Unknown'").count()
# There are 384 customers with Unknown Country

In [19]:
print(f'There are {Unknown} customers with Unknown Country')

There are 384 customers with Unknown Country


In [20]:
df_emp.write.csv('cleaned_churn_location_and_rating.csv', header=True)
